# Ingesta y Capa Bronce

En esta notebook se inicia la construcción del pipeline de datos meteorológicos, trabajando con los archivos crudos provistos por el SMN.


## Importar las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import json
from glob import glob
from pathlib import Path

print("Importación de librerías completada.")

## Configuración de paths y carpetas del proyecto

In [ ]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'

# Crear carpetas si no existen
for path in [BRONCE_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print("Iniciación de carpetas del proyecto completada.")

## Lectura del archivo de estaciones

In [ ]:
# Ruta del archivo
archivo_estaciones = RAW_DIR / 'estaciones' / 'estaciones_smn.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_estaciones, "r", encoding="latin1") as f:
    lines = f.readlines()[2:]

# Expresión regular para extraer campos:
# Modifico la expresión porque tenía algunos problemas:
# 1. Una de las estaciones de jujuy no tiene código OACI y la expresión ignoraba la linea. Para resolverlo hago la extracción del código oaci opcional.
# 2. Hay dos estaciones con nombres largos y entonces la separación entre nombre y provincia es menor a 2 caracteres. Para resolverlo, como el archivo esta organizado en columnas fijas, uso la columna 31 como límite entre nombre y provincia.
# Con estos cambios, ahora sí la cantidad de estaciones se carga correctamente (118) y las provincias se cuentan como 25 (las 23 + CABA y antártida), en lugar de 26 provincias y 117 estaciones de antes.

pattern = re.compile(
    r"^(?P<nombre>.{31})(?P<provincia>.+?)\s{2,}(?P<lat_gr>-?\d+)\s+(?P<lat_min>\d+)\s+(?P<lon_gr>-?\d+)\s+(?P<lon_min>\d+)\s+(?P<altura_m>\d+)\s+(?P<numero>\d+)(?:\s+(?P<numero_oaci>\S+))?\s*$"
)

# Extraer los datos
data = []
for line in lines:
    match = pattern.match(line)
    if match:
        data.append(match.groupdict())

# Crear DataFrame
df_estaciones = pd.DataFrame(data)

# Conversión de tipos
df_estaciones[['lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero']] = df_estaciones[[
    'lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero'
]].apply(pd.to_numeric)

# Cargar las provincias
provincias_unicas = df_estaciones['provincia'].str.strip().str.upper().unique()

#En los archivos de datos horarios el nombre de algunas estaciones está truncado en 26 caracteres y los demás pasan a la linea siguiente.
#Esto hace que esas escaciones queden sin detectar ya que se realiza una comparación de strings que no son iguales. Justo córdoba (la provincia que elegí) tiene dos estaciones de nombres largos
# que quedan sin detectar. Para resolver esto trunco los nombres de las estaciones a 26 caracteres. No tiene efectos adversos porque esa cantidad de caracteres es suficiente para seguir teniendo nombres únicos en cada estación.

df_estaciones['nombre'] = df_estaciones['nombre'].str.slice(0, 26)

# Imprimir la cantidad de estaciones registradas
print("Estaciones cargadas:", len(df_estaciones))

# Imprimir la cantidad de provincias registradas
print("Cantidad de provincias:", len(provincias_unicas))

# Imprimir las provincias
print("Provincias disponibles:", provincias_unicas)

In [ ]:
df_estaciones.iloc[45:55]

# Trato de geolocalizar las estaciones

In [ ]:
def dms_to_decimal(deg, minutes):
    sign = 1 if deg >= 0 else -1
    return sign * (abs(deg) + minutes / 60)

df_estaciones["LAT_DEC"] = df_estaciones.apply(lambda row: dms_to_decimal(row["lat_gr"], row["lat_min"]), axis=1)
df_estaciones["LON_DEC"] = df_estaciones.apply(lambda row: dms_to_decimal(row["lon_gr"], row["lon_min"]), axis=1)
df_estaciones[["LAT_DEC", "LON_DEC"]].to_csv(BRONCE_DIR / 'coordenadas_estaciones.csv', index=False)
df_estaciones.head()

## Selección de estaciones. 

### Para el desarrollo del trabajo se utilizarán las estaciones ubicadas en la provincia de Córdoba

In [ ]:
# Ingresar el nombre de la provincia con la que se va a trabajar
provincia = 'CORDOBA'

df_provincia = df_estaciones[df_estaciones['provincia'].str.upper() == provincia]
df_provincia[['nombre', 'provincia', 'numero', 'numero_oaci']]

## Filtrar las estaciones que correspondan a la provincia seleccionada

In [ ]:
# Se selecciona una fecha para visualizar los datos
archivo_dato = RAW_DIR / 'datohorario' /  '_procesados' / 'datohorario20250531.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_dato, "r", encoding="latin1") as f:
    lines = f.readlines()

# Detectar columnas separadas por múltiples espacios
columnas = re.split(r"\s{2,}", lines[0].strip())

# Leer datos
data = [
    re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
    for line in lines[1:]
    if len(line.strip()) > 0 and not line.isspace()
]

# Crear DataFrame con columnas originales
df_dato = pd.DataFrame(data, columns=columnas)
df_dato.columns = df_dato.columns.str.strip()
df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()

# Filtrar por estaciones
nombres_provincia = df_provincia["nombre"].str.strip().unique()
df_provincia_dia = df_dato[df_dato["NOMBRE"].isin(nombres_provincia)]

# Crear copia y convertir tipos SOLO para impresión de tipos correctos
df_tipos = df_provincia_dia.copy()
df_tipos["FECHA"] = pd.to_datetime(df_tipos["FECHA"], format="%d%m%Y", errors="coerce").dt.date
df_tipos["HORA"] = pd.to_numeric(df_tipos["HORA"], errors="coerce").astype("Int64")
df_tipos["TEMP"] = pd.to_numeric(df_tipos["TEMP"], errors="coerce")
df_tipos["HUM"] = pd.to_numeric(df_tipos["HUM"], errors="coerce")
df_tipos["PNM"] = pd.to_numeric(df_tipos["PNM"], errors="coerce")
df_tipos["DD"] = pd.to_numeric(df_tipos["DD"], errors="coerce").astype("Int64")
df_tipos["FF"] = pd.to_numeric(df_tipos["FF"], errors="coerce").astype("Int64")

# Mostrar todos los resultados
print(df_provincia_dia.to_string(index=False))
print()
print("Columnas:", df_dato.columns.tolist())
print("Tipos de dato:")
print(df_tipos.dtypes)


## Procesamiento por estación y por fecha (con limpieza y reporte resumen)

In [ ]:
# Crear carpeta de salida si no existe
BRONCE_DIR.mkdir(parents=True, exist_ok=True)

# Buscar todos los archivos datohorario disponibles
archivos_datos = sorted(glob(str(RAW_DIR / "datohorario" / '_procesados' / "datohorario*.txt")))

errores_globales = 0

for archivo in archivos_datos:
    try:
        with open(archivo, encoding="latin1") as f:
            raw_lines = f.readlines()

        header = raw_lines[0].strip()
        columnas = re.split(r"\s{2,}", header)

        data = [
            re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
            for line in raw_lines[1:]
            if len(line.strip()) > 0 and not line.isspace()
        ]

        df_dato = pd.DataFrame(data, columns=columnas)
        df_dato.columns = df_dato.columns.str.strip()
        df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()

        # Filtrar por estaciones según la provincia
        df_provincia = df_dato[df_dato["NOMBRE"].isin(nombres_provincia)]

        # Obtener fecha
        fecha = Path(archivo).stem.replace("datohorario", "")

        # Guardar archivos por estación
        for nombre in nombres_provincia:
            nombre_clean = nombre.lower().replace(" ", "_")
            df_estacion = df_provincia[df_provincia["NOMBRE"] == nombre]

            if not df_estacion.empty:
                path_estacion = BRONCE_DIR / nombre_clean
                path_estacion.mkdir(parents=True, exist_ok=True)

                # Archivos de salida
                archivo_csv = path_estacion / f"{fecha}.csv"
                df_estacion.to_csv(archivo_csv, index=False)

    except Exception as e:
        errores_globales += 1
        continue

# Reporte final
print("Proceso completado.")
print(f"Días procesados: {len(archivos_datos)}")
print(f"Errores al procesar archivos: {errores_globales}")

# Conclusión

En este notebook realizamos el proceso de **ingesta de datos meteorológicos** y la creación de la **Capa Bronce** de nuestro proyecto:

1. **Lectura de datos crudos** provenientes de archivos del Servicio Meteorológico Nacional.
2. **Estructuración inicial de datos**, manteniendo la información tal como llega del entorno real, sin limpieza ni transformación.
3. **Organización en la estructura de carpetas** del proyecto, asegurando que los datos queden almacenados en la capa correspondiente para futuras etapas de procesamiento.

Esta etapa constituye la **base del pipeline de datos**, preservando la trazabilidad y sirviendo como fuente de verdad para las capas posteriores (Plata y Oro).